In [1]:
import numpy as np
import csv
import caffe
from caffe.proto.caffe_pb2 import NetParameter, LayerParameter
import google.protobuf.text_format as txtf
from pci import pci
from sktensor import dtensor
from argparse import ArgumentParser
import argparse

In [4]:
# load parameters for btd
def load_config(config_file):
    btd_config = {}
    with open(config_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            param, s, t, blocks = row[0], int(row[1]), int(row[2]), int(row[3])
            btd_config[param] = (s, t, blocks)
    return btd_config


def create_approx_netdef(input_file, output_file, btd_config):
    with open(input_file, 'r') as fp:
        net = NetParameter()
        txtf.Merge(fp.read(), net)
    new_layers = []
    for layer in net.layer:
        if layer.name not in list(btd_config.keys()):
            new_layers.append(layer)
            continue
        s, t, r = btd_config[layer.name]
        a, b, c = decompose2abc(layer, s, t, r)
        new_layers.extend([a, b, c])
    new_net = NetParameter()
    new_net.CopyFrom(net)
    del(new_net.layer[:])
    new_net.layer.extend(new_layers)
    with open(output_file, 'w') as fp:
        fp.write(txtf.MessageToString(new_net))


def decompose2abc(conv, s, t, r):
    print(conv.name)

    def _create_new(name):
        print("--> {0}".format(name))
        new_ = LayerParameter()
        new_.CopyFrom(conv)
        new_.name = name
        return new_
    # 1st
    a = _create_new(conv.name + 'a')
    del(a.top[:])
    a.top.extend([a.name])
    if len(a.param) > 2:
        a.param[1].lr_mult = 0
    a_param = a.convolution_param
    a_param.num_output = s
    del(a_param.kernel_size[:])
    a_param.kernel_size.extend([1])
    del(a_param.pad[:])
    a_param.pad.extend([0])
    del(a_param.stride[:])
    a_param.stride.extend([1])
    # 2nd
    b = _create_new(conv.name + 'b')
    del(b.bottom[:])
    b.bottom.extend(a.top)
    del(b.top[:])
    b.top.extend([b.name])
    if len(b.param) > 2:
        b.param[1].lr_mult = 0
    b_param = b.convolution_param
    b_param.num_output = t
    b_param.group = r
    # 3rd
    c = _create_new(conv.name + 'c')
    del(c.bottom[:])
    c.bottom.extend(b.top)
    c_param = c.convolution_param
    del(c_param.kernel_size[:])
    c_param.kernel_size.extend([1])
    del(c_param.pad[:])
    c_param.pad.extend([0])
    del(c_param.stride[:])
    c_param.stride.extend([1])
    return a, b, c


def approximate_params(netdef, params, approx_netdef, approx_params,
                       btd_config, max_iter, min_decrease):
    net = caffe.Net(netdef, params, caffe.TEST)
    net_approx = caffe.Net(approx_netdef, params, caffe.TEST)
    convs = [(k, v[0].data, v[1].data) for k, v in list(
        net.params.items()) if k in list(btd_config.keys())]
    for conv, kernel, bias in convs:
        size = kernel.shape
        T, S, H, W = size[0:4]
        P = H * W
        s, t, blocks = btd_config[conv]
        # (T, S, H, W) -> (T, S, P)
        kernel = kernel.reshape(T, S, P)
        # compute BTD
        t_ = int(t/blocks)
        s_ = int(s/blocks)
        rank = [t_, s_, P]
        print(('calculating BTD for {0}...').format(conv))
        btd, _ = pci(dtensor(kernel), blocks, rank, max_iter, min_decrease)
        # BTD -> (c, C) (n, c/blocks, P) (N, n)
        kernel_a = np.concatenate([subtensor[1][1]
                                   for subtensor in btd], axis=1)
        kernel_b = np.concatenate(
            [subtensor[0].ttm(subtensor[1][2], 2) for subtensor in btd], axis=0)
        kernel_c = np.concatenate([subtensor[1][0]
                                   for subtensor in btd], axis=1)
        # (c, C) -> (c, C, 1, 1)
        kernel_a = kernel_a.T.reshape(s, S, 1, 1)
        # (n, c/blocks, P) -> (n, c/blocks, H, W)
        kernel_b = kernel_b.reshape(t, s_, H, W)
        # (N, n) -> (N, n, 1, 1)
        kernel_c = kernel_c.reshape(T, t, 1, 1)
        # set kernel to low-rank model
        net_approx.params[conv + 'a'][0].data[...] = kernel_a
        net_approx.params[conv + 'b'][0].data[...] = kernel_b
        net_approx.params[conv + 'c'][0].data[...] = kernel_c
        # copy bias to low-rank model
        net_approx.params[conv + 'c'][1].data[...] = bias
    net_approx.save(approx_params)

In [ ]:
def main(args):
    btd_config = load_config(args.config)
    create_approx_netdef(args.netdef, args.save_netdef, btd_config)

    if args.params is None:
        return
    if args.max_iter is None:
        max_iter = 100
    else:
        max_iter = int(args.max_iter)
    if args.min_decrease is None:
        min_decrease = 1e-5
    else:
        min_decrease = float(args.min_decrease)

    approximate_params(args.netdef, args.params, args.save_netdef, args.save_params,
                       btd_config, max_iter, min_decrease)


if __name__ == '__main__':
    '''
    parser = ArgumentParser(
        description="Block Term Decomposition on Convolution Kernel")
    parser.add_argument('--netdef', required=True,
                        help="Prototxt of the original net")
    parser.add_argument('--save_netdef', required=True,
                        help="Path to the deploy.prototxt of the low-rank approximated net")
    parser.add_argument('--config', required=True,
                        help="CSV config file for BTD")
    parser.add_argument('--params',
                        help="Caffemodel of the original net")
    parser.add_argument('--save_params',
                        help="Path to the caffemodel of the low-rank approximated net")
    parser.add_argument('--max_iter',
                        help="Max iteration for BTD")
    parser.add_argument('--min_decrease',
                        help="Minimum error decrease in each iteration for BTD")
    args = parser.parse_args()
    main(args)
    '''
    def create_approx():
        d = {
            'netdef': 'vgg16/deploy.prototxt',
            'save_netdef': 'vgg16/lowrank/deploy.prototxt',
            'config': 'vgg16/params.csv',
            'params': None
        }
        args = argparse.Namespace(**d)
        main(args)

    def create_approx_and_approximate_params():
        d = {
            'netdef': 'vgg16/deploy.prototxt',
            'save_netdef': 'vgg16/lowrank/deploy.prototxt',
            'config': 'vgg16/params.csv',
            'params': 'vgg16/vgg16.caffemodel',
            'save_params': 'vgg16/lowrank/vgg16_lowrank.caffemodel',
            'max_iter': 1000,
            'min_decrease': 1e-5,
        }
        args = argparse.Namespace(**d)
        main(args)

    create_approx_and_approximate_params()

conv1_2
--> conv1_2a
--> conv1_2b
--> conv1_2c
conv2_1
--> conv2_1a
--> conv2_1b
--> conv2_1c
conv2_2
--> conv2_2a
--> conv2_2b
--> conv2_2c
conv3_1
--> conv3_1a
--> conv3_1b
--> conv3_1c
conv3_2
--> conv3_2a
--> conv3_2b
--> conv3_2c
conv3_3
--> conv3_3a
--> conv3_3b
--> conv3_3c
conv4_1
--> conv4_1a
--> conv4_1b
--> conv4_1c
conv4_2
--> conv4_2a
--> conv4_2b
--> conv4_2c
conv4_3
--> conv4_3a
--> conv4_3b
--> conv4_3c
conv5_1
--> conv5_1a
--> conv5_1b
--> conv5_1c
conv5_2
--> conv5_2a
--> conv5_2b
--> conv5_2c
conv5_3
--> conv5_3a
--> conv5_3b
--> conv5_3c
calculating BTD for conv1_2...
iter 1
	 HOOI 1
		 norm 7.88325444089
	 --------------------
	 Error 0.279356139232
	 Decrease 0.720643860768
iter 2
	 HOOI 1
		 norm 7.88325444089
	 --------------------
	 Error 0.279356139232
	 Decrease 0.0
calculating BTD for conv2_1...
iter 1
	 HOOI 1
		 norm 7.64629867122
	 --------------------
	 Error 0.486113661697
	 Decrease 0.513886338303
iter 2
	 HOOI 1
		 norm 7.64629867122
	 ---------------

		 norm 6.2591327353
	 HOOI 2
		 norm 5.51552563877
	 --------------------
	 Error 0.590660845359
	 Decrease 3.20816543571e-05
iter 53
	 HOOI 1
		 norm 6.27396349948
	 HOOI 2
		 norm 5.53356095797
	 --------------------
	 Error 0.590629246174
	 Decrease 3.15991845307e-05
iter 54
	 HOOI 1
		 norm 6.28881175519
	 HOOI 2
		 norm 5.55159229446
	 --------------------
	 Error 0.590598115618
	 Decrease 3.11305569575e-05
iter 55
	 HOOI 1
		 norm 6.30367567096
	 HOOI 2
		 norm 5.56961800218
	 --------------------
	 Error 0.590567441405
	 Decrease 3.06742128541e-05
iter 56
	 HOOI 1
		 norm 6.31855339903
	 HOOI 2
		 norm 5.58763634506
	 --------------------
	 Error 0.590537212592
	 Decrease 3.0228812787e-05
iter 57
	 HOOI 1
		 norm 6.33344308042
	 HOOI 2
		 norm 5.6056455131
	 --------------------
	 Error 0.590507419382
	 Decrease 2.97932098252e-05
iter 58
	 HOOI 1
		 norm 6.34834284991
	 HOOI 2
		 norm 5.62364363748
	 --------------------
	 Error 0.590478052956
	 Decrease 2.93664260589e-05
iter 

		 norm 6.49019038701
	 --------------------
	 Error 0.589422529896
	 Decrease 1.43241653822e-05
iter 110
	 HOOI 1
		 norm 7.10042863019
	 HOOI 2
		 norm 6.50575961166
	 --------------------
	 Error 0.589408386423
	 Decrease 1.41434734628e-05
iter 111
	 HOOI 1
		 norm 7.11401459621
	 HOOI 2
		 norm 6.52126935805
	 --------------------
	 Error 0.589394420001
	 Decrease 1.39664214184e-05
iter 112
	 HOOI 1
		 norm 7.12756005734
	 HOOI 2
		 norm 6.5367197979
	 --------------------
	 Error 0.589380627057
	 Decrease 1.37929444258e-05
iter 113
	 HOOI 1
		 norm 7.14106497947
	 HOOI 2
		 norm 6.55211112009
	 --------------------
	 Error 0.589367004079
	 Decrease 1.36229776208e-05
iter 114
	 HOOI 1
		 norm 7.15452934434
	 HOOI 2
		 norm 6.56744352962
	 --------------------
	 Error 0.589353547623
	 Decrease 1.34564562774e-05
iter 115
	 HOOI 1
		 norm 7.16795314876
	 HOOI 2
		 norm 6.58271724661
	 --------------------
	 Error 0.589340254307
	 Decrease 1.32933158608e-05
iter 116
	 HOOI 1
		 norm 7.

		 norm 4.07166090103
	 HOOI 3
		 norm 3.47008556813
	 HOOI 4
		 norm 3.70057238311
	 --------------------
	 Error 0.582259903974
	 Decrease 0.000444268154701
iter 18
	 HOOI 1
		 norm 5.42428654704
	 HOOI 2
		 norm 4.08352253276
	 HOOI 3
		 norm 3.48845514488
	 HOOI 4
		 norm 3.74561145781
	 --------------------
	 Error 0.581839366126
	 Decrease 0.000420537848169
iter 19
	 HOOI 1
		 norm 5.4439787169
	 HOOI 2
		 norm 4.09509880779
	 HOOI 3
		 norm 3.50701052275
	 HOOI 4
		 norm 3.78848117951
	 --------------------
	 Error 0.581442449358
	 Decrease 0.000396916767421
iter 20
	 HOOI 1
		 norm 5.46342519667
	 HOOI 2
		 norm 4.10638426539
	 HOOI 3
		 norm 3.52561928389
	 HOOI 4
		 norm 3.82927467971
	 --------------------
	 Error 0.581068744553
	 Decrease 0.000373704805214
iter 21
	 HOOI 1
		 norm 5.48259482145
	 HOOI 2
		 norm 4.11737496049
	 HOOI 3
		 norm 3.54416657596
	 HOOI 4
		 norm 3.86810760965
	 --------------------
	 Error 0.580717301017
	 Decrease 0.000351443536035
iter 22
	 HOOI

		 norm 6.03523031444
	 HOOI 2
		 norm 4.41107279756
	 HOOI 3
		 norm 4.00085737095
	 HOOI 4
		 norm 4.75783758061
	 --------------------
	 Error 0.573894728045
	 Decrease 0.000112717254055
iter 58
	 HOOI 1
		 norm 6.04953499083
	 HOOI 2
		 norm 4.41819150292
	 HOOI 3
		 norm 4.00902169008
	 HOOI 4
		 norm 4.77701893295
	 --------------------
	 Error 0.57378355513
	 Decrease 0.000111172915109
iter 59
	 HOOI 1
		 norm 6.06383569981
	 HOOI 2
		 norm 4.42527936281
	 HOOI 3
		 norm 4.01705943893
	 HOOI 4
		 norm 4.79602568221
	 --------------------
	 Error 0.573673770587
	 Decrease 0.000109784542685
iter 60
	 HOOI 1
		 norm 6.07813210916
	 HOOI 2
		 norm 4.43233932674
	 HOOI 3
		 norm 4.02497560241
	 HOOI 4
		 norm 4.81486807152
	 --------------------
	 Error 0.573565271105
	 Decrease 0.000108499482416
iter 61
	 HOOI 1
		 norm 6.09242377182
	 HOOI 2
		 norm 4.43937439961
	 HOOI 3
		 norm 4.03277372246
	 HOOI 4
		 norm 4.83355167256
	 --------------------
	 Error 0.573458038146
	 Decrease 0

...

		 norm 11.4599287023
	 HOOI 2
		 norm 9.7841250336
	 HOOI 3
		 norm 9.67647466877
	 HOOI 4
		 norm 9.66713714231
	 --------------------
	 Error 0.584519000532
	 Decrease 1.35213231675e-05
iter 202
	 HOOI 1
		 norm 11.470204851
	 HOOI 2
		 norm 9.79460215161
	 HOOI 3
		 norm 9.68767703548
	 HOOI 4
		 norm 9.6794164685
	 --------------------
	 Error 0.584505574779
	 Decrease 1.3425752682e-05
iter 203
	 HOOI 1
		 norm 11.4804474492
	 HOOI 2
		 norm 9.80504659774
	 HOOI 3
		 norm 9.69884407652
	 HOOI 4
		 norm 9.6916545027
	 --------------------
	 Error 0.584492243501
	 Decrease 1.33312785112e-05
iter 204
	 HOOI 1
		 norm 11.4906567754
	 HOOI 2
		 norm 9.8154586127
	 HOOI 3
		 norm 9.70997608413
	 HOOI 4
		 norm 9.70385156687
	 --------------------
	 Error 0.584479005616
	 Decrease 1.32378847417e-05
iter 205
	 HOOI 1
		 norm 11.5008331055
	 HOOI 2
		 norm 9.82583843422
	 HOOI 3
		 norm 9.72107334659
	 HOOI 4
		 norm 9.71600797919
	 --------------------
	 Error 0.58446586006
	 Decrease 1.3

		 norm 11.8473057411
	 HOOI 2
		 norm 10.1798614935
	 HOOI 3
		 norm 10.099585103
	 HOOI 4
		 norm 10.1289669018
	 --------------------
	 Error 0.584046258598
	 Decrease 1.04181466541e-05
iter 242
	 HOOI 1
		 norm 11.8564304499
	 HOOI 2
		 norm 10.1891941284
	 HOOI 3
		 norm 10.109569166
	 HOOI 4
		 norm 10.1398122952
	 --------------------
	 Error 0.58403590231
	 Decrease 1.03562874945e-05
iter 243
	 HOOI 1
		 norm 11.8655307086
	 HOOI 2
		 norm 10.1985018219
	 HOOI 3
		 norm 10.1195271068
	 HOOI 4
		 norm 10.1506267466
	 --------------------
	 Error 0.584025607253
	 Decrease 1.02950569217e-05
iter 244
	 HOOI 1
		 norm 11.8746066975
	 HOOI 2
		 norm 10.2077847271
	 HOOI 3
		 norm 10.1294591031
	 HOOI 4
		 norm 10.161410459
	 --------------------
	 Error 0.584015372807
	 Decrease 1.02344468559e-05
iter 245
	 HOOI 1
		 norm 11.883658595
	 HOOI 2
		 norm 10.2170429957
	 HOOI 3
		 norm 10.1393653305
	 HOOI 4
		 norm 10.172163633
	 --------------------
	 Error 0.584005198358
	 Decrease 1.

		 norm 8.91899732759
	 HOOI 2
		 norm 6.83637573723
	 HOOI 3
		 norm 6.43001640279
	 HOOI 4
		 norm 6.14450031339
	 --------------------
	 Error 0.578620954191
	 Decrease 0.000144409306311
iter 34
	 HOOI 1
		 norm 8.94559089848
	 HOOI 2
		 norm 6.8666883128
	 HOOI 3
		 norm 6.46496778957
	 HOOI 4
		 norm 6.18246501589
	 --------------------
	 Error 0.578482360901
	 Decrease 0.000138593290356
iter 35
	 HOOI 1
		 norm 8.97174031916
	 HOOI 2
		 norm 6.8964508187
	 HOOI 3
		 norm 6.49927590515
	 HOOI 4
		 norm 6.21968490867
	 --------------------
	 Error 0.578349189956
	 Decrease 0.000133170945031
iter 36
	 HOOI 1
		 norm 8.99746501421
	 HOOI 2
		 norm 6.9256865344
	 HOOI 3
		 norm 6.53297112317
	 HOOI 4
		 norm 6.256195284
	 --------------------
	 Error 0.578221084146
	 Decrease 0.000128105809279
iter 37
	 HOOI 1
		 norm 9.02278307067
	 HOOI 2
		 norm 6.95441737282
	 HOOI 3
		 norm 6.56608153888
	 HOOI 4
		 norm 6.2920288378
	 --------------------
	 Error 0.578097718473
	 Decrease 0.0001

		 norm 9.75345859268
	 HOOI 2
		 norm 7.76787014776
	 HOOI 3
		 norm 7.50429348045
	 HOOI 4
		 norm 7.28993515553
	 --------------------
	 Error 0.575388959449
	 Decrease 4.89597149633e-05
iter 74
	 HOOI 1
		 norm 9.77024719
	 HOOI 2
		 norm 7.78633241172
	 HOOI 3
		 norm 7.52554263853
	 HOOI 4
		 norm 7.31216184798
	 --------------------
	 Error 0.575340885627
	 Decrease 4.80738220174e-05
iter 75
	 HOOI 1
		 norm 9.78689799473
	 HOOI 2
		 norm 7.80463906126
	 HOOI 3
		 norm 7.54660633246
	 HOOI 4
		 norm 7.33417979391
	 --------------------
	 Error 0.575293669887
	 Decrease 4.72157399177e-05
iter 76
	 HOOI 1
		 norm 9.80341376262
	 HOOI 2
		 norm 7.82279346862
	 HOOI 3
		 norm 7.56748834593
	 HOOI 4
		 norm 7.35599354479
	 --------------------
	 Error 0.575247285667
	 Decrease 4.63842194793e-05
iter 77
	 HOOI 1
		 norm 9.81979716894
	 HOOI 2
		 norm 7.84079888068
	 HOOI 3
		 norm 7.58819233263
	 HOOI 4
		 norm 7.37760750377
	 --------------------
	 Error 0.575201707583
	 Decrease 4.5

		 norm 10.3390217628
	 HOOI 2
		 norm 8.40985942104
	 HOOI 3
		 norm 8.23819110642
	 HOOI 4
		 norm 8.0509303288
	 --------------------
	 Error 0.573948732578
	 Decrease 2.72618786371e-05
iter 114
	 HOOI 1
		 norm 10.3518636057
	 HOOI 2
		 norm 8.4238818399
	 HOOI 3
		 norm 8.25409262406
	 HOOI 4
		 norm 8.06732595196
	 --------------------
	 Error 0.573921790569
	 Decrease 2.69420095728e-05
iter 115
	 HOOI 1
		 norm 10.3646359058
	 HOOI 2
		 norm 8.43782463877
	 HOOI 3
		 norm 8.2698984991
	 HOOI 4
		 norm 8.0836221182
	 --------------------
	 Error 0.573895161918
	 Decrease 2.66286510757e-05
iter 116
	 HOOI 1
		 norm 10.3773396325
	 HOOI 2
		 norm 8.45168884397
	 HOOI 3
		 norm 8.28561002583
	 HOOI 4
		 norm 8.09982038791
	 --------------------
	 Error 0.5738688403
	 Decrease 2.63216181235e-05
iter 117
	 HOOI 1
		 norm 10.3899757326
	 HOOI 2
		 norm 8.46547546004
	 HOOI 3
		 norm 8.30122847173
	 HOOI 4
		 norm 8.11592228724
	 --------------------
	 Error 0.573842819567
	 Decrease 2.

# caffe2 playground

In [ ]:
x = np.random.randn(2, 3).astype(np.float32)
print("Current blobs in the workspace: {}".format(workspace.Blobs()))
print("Workspace has blob 'X'? {}".format(workspace.HasBlob("X")))

workspace.FeedBlob('X', x)

In [ ]:
x

In [ ]:
workspace.FetchBlob('X')